In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
from shutil import copy2

In [3]:
import matplotlib.pyplot as plt

In [4]:
races = ['African','Asian','Caucasian','Indian']
labels = []
dist = []
for race in races:
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.4119390845298767


In [6]:
races = ['African','Asian','Caucasian','Indian']
accuracies = {}
for race in races:
        accuracies[f"{race}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TN'] += 1
                else:
                    accuracies[f"{race}"]['FN'] += 1
            else:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TP'] += 1
                else:
                    accuracies[f"{race}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African': {'TP': 2405, 'TN': 595, 'FP': 597, 'FN': 2403}, 'Asian': {'TP': 2386, 'TN': 614, 'FP': 662, 'FN': 2338}, 'Caucasian': {'TP': 2141, 'TN': 859, 'FP': 59, 'FN': 2941}, 'Indian': {'TP': 2450, 'TN': 550, 'FP': 429, 'FN': 2570}}


In [9]:
gar = {}
far = {}
scores = {}
for race in races:
        tp = accuracies[f"{race}"]['TP']
        tn = accuracies[f"{race}"]['TN']
        fp = accuracies[f"{race}"]['FP']
        fn = accuracies[f"{race}"]['FN']
        gar[f"{race}"] = tp/(tn+tp)
        far[f"{race}"] = fp/(fn+fp)
        scores[f"{race}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [10]:
gar

{'African': 0.8016666666666666,
 'Asian': 0.7953333333333333,
 'Caucasian': 0.7136666666666667,
 'Indian': 0.8166666666666667}

In [11]:
far

{'African': 0.199,
 'Asian': 0.22066666666666668,
 'Caucasian': 0.019666666666666666,
 'Indian': 0.14304768256085362}

In [12]:
scores

{'African': 80.13333333333334,
 'Asian': 78.73333333333333,
 'Caucasian': 84.7,
 'Indian': 83.6806134355726}